In [10]:
import numpy as np
import random
import copy
import cv2
class AntColonySegmentation:
    def __init__(self, image, num_ants, max_iterations, alpha, beta, rho):
        self.image = image
        self.num_ants = num_ants
        self.max_iterations = max_iterations
        self.alpha = alpha  # pheromone importance
        self.beta = beta  # visibility (intensity) importance
        self.rho = rho  # pheromone evaporation rate
        self.green_found = True

        self.pheromones = np.ones([self.image.shape[0],self.image.shape[1]])

    def initialize_ants(self):
        ants = []
        for _ in range(self.num_ants):
            ant = {'path': [], 'intensity_sum': 0.0}
            ants.append(ant)
        return ants

    
    def clear_ant_paths(self, ants):
        for ant in ants:
            ant['path'] = []
            ant['intensity_sum'] = 0.0

    # def dead_end(self,neighbors):
    #     #neighbors = self.get_neighbors(pos)
    #     for n in neighbors:
    #         if self.image[n][1] > self.image[n][2] and self.image[n][1] > self.image[n][0]:
    #             return False
    #     return True

    def construct_segment(self, ant):
        current_position = (np.random.randint(0, self.image.shape[0]), np.random.randint(0, self.image.shape[1]))
        ant['path'].append(current_position)
        
        briter = 0
        b = 0
        self.green_found = True
        while self.green_found: # and briter<5000:
            briter += 1
            neighbors = self.get_neighbors(current_position)
            probabilities = self.calculate_probabilities(current_position,neighbors,ant)

            i = 2  #backtracking
            length = len(ant['path'])
            while sum(probabilities) == 0 and i <= length:
                current_position = ant['path'][-i]
                neighbors = self.get_neighbors(current_position)
                probabilities = self.calculate_probabilities(current_position,neighbors,ant)
                i += 1

            if i == length and length > 2:
                self.green_found = False
                print('i==length')
                break

            if sum(probabilities) == 0:
                self.green_found = False
                break
            
            pom = [_ for _ in range(len(neighbors))]
            next_position = np.random.choice(pom, p=probabilities)
            next_position = neighbors[next_position]
            
            if self.image[next_position][1] < self.image[next_position][2] or self.image[next_position][1] < self.image[next_position][0]:
                b += 1
            else:
                b = 0

            if b==50:
                self.green_found = False
                #print('green not found')
                break
            
            if next_position not in ant['path']:
                ant['path'].append(next_position)
                current_position = next_position
        #print(briter)
                

    def get_neighbors(self, position):
        height, width,_ = self.image.shape
        row, col = position
        neighbors = []
        #neighbors_offsets = [(0, 1), (0, -1),(1,0),(-1,0)]
        neighbors_offsets = [(0, 1), (0, -1), (1, 0), (-1, 0),(1,1),(1,-1),(-1,-1),(-1,1)]

        for offset_row, offset_col in neighbors_offsets:
            new_row, new_col = row + offset_row, col + offset_col
            if 0 <= new_row < self.image.shape[0] and 0 <= new_col < self.image.shape[1]:
                neighbors.append((new_row, new_col))

        return neighbors

    def calculate_probabilities(self, current_position,neighbors,ant):
        intensity = self.image[current_position][1]
        probabilities = []

        # if self.dead_end(neighbors):
        #     probabilities = [0 for _ in range(len(neighbors))]
        #     return probabilit
        
        for neighbor in neighbors:
            attractiveness = self.calc_attractiveness(neighbor)
            #neighbor_intensity = self.image[neighbor][1]
            pheromone = self.pheromones[neighbor]
            probability = (pheromone ** self.alpha) * (attractiveness ** self.beta)
            #if probability == float('inf'):
                #probability = 10000.0
            if neighbor in ant['path']:
                probability = 0
                #print([n in self.visited for n in neighbors])
            probabilities.append(probability)
            
            

        total_probability = sum(probabilities)
        #probabilities = [p / total_probability for p in probabilities]
        if total_probability > 0:
            probabilities /= total_probability
            
        
        return probabilities

    def calc_attractiveness(self,position):
        row, col = position
        pixel = self.image[row, col]
        if pixel[1] > pixel[2] and pixel[1] > pixel[2]:
            return 0.9  # High attractiveness for green pixels
        else:
            return 0.1

    def update_pheromones(self, ants):
        self.pheromones *= (1 - self.rho)  # Evaporation
        for ant in ants:
            for position in ant['path']:
                self.pheromones[position] += ant['intensity_sum']  #sklonio 1.0/

    def run(self):
        ants = self.initialize_ants()
        best_ant = None
        segmentation_result = np.zeros_like(self.image)
        iter=0
        
        for iteration in range(self.max_iterations):
            iter+=1
            self.clear_ant_paths(ants)
            #print('suma feromona',np.sum(self.pheromones))
            for ant in ants:
                self.construct_segment(ant)
                #ant['intensity_sum'] = sum(self.image[position][1] for position in ant['path'])
                ant['intensity_sum'] = sum((self.image[position][1]>self.image[position][0] and self.image[position][1]>self.image[position][2]) \
                                           for position in ant['path'])
                

            self.update_pheromones(ants)
            

            # Choose the best segment (ant) based on intensity sum
            current_best_ant = max(ants, key=lambda ant: ant['intensity_sum'])
            if best_ant is None or current_best_ant['intensity_sum'] > best_ant['intensity_sum']:
                best_ant = copy.deepcopy(current_best_ant)

            print(iter,'len best path',len(current_best_ant['path']))
        
        #segmentation_result = np.zeros_like(self.image)
            for position in current_best_ant['path']:
                segmentation_result[position] = self.image[position]

        return segmentation_result


In [11]:
#image = cv2.imread("img/2.jpg")
image = cv2.imread("img/8s.png")

print(image.shape)

num_ants = 20
max_iterations = 10

alpha = 0.9
beta = 0.9
rho = 0.1

aco_segmentation = AntColonySegmentation(image, num_ants, max_iterations, alpha, beta, rho)

result = aco_segmentation.run()

cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

(333, 500, 3)
1 len best path 2299
2 len best path 1470
3 len best path 2546
4 len best path 2335
5 len best path 2062
6 len best path 4057
7 len best path 2588
8 len best path 2221
9 len best path 1805
10 len best path 2346


In [6]:
import cv2
import numpy as np
image = cv2.imread("img/1s.png")
path = []
visited = []
b = 0
briter=0
def get_neighbors(position):
        height, width,_ = image.shape
        row, col = position
        neighbors = []
        neighbors_offsets = [(0, 1), (0, -1), (1, 0), (-1, 0),(1,1),(1,-1),(-1,-1),(-1,1)]
        #neighbors_offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for offset_row, offset_col in neighbors_offsets:
            new_row, new_col = row + offset_row, col + offset_col
            if 0 <= new_row < image.shape[0] and 0 <= new_col < image.shape[1]:
                neighbors.append((new_row, new_col))
        return neighbors

def max_y(lst):
    # Return the pair with the maximum x and y coordinates
    return max(lst, key=lambda pair: (pair[1]))

def difff(x,y):
    if x<y:
        return y-x
    else:
        return x-y

def segmentation():
        current_position = (0,0)
        #current_position = (np.random.randint(0, image.shape[0]), np.random.randint(0, image.shape[1]))
        path.append(current_position)
        visited.append(current_position)

        green_found = True

        global briter
        while green_found and briter<2000:
            briter+=1
            neighbors = get_neighbors(current_position)
            #print(neighbors)

            next = None
            min = float('inf')
            for neighbor in neighbors:
                #print(image[neighbor][1],image[current_position][1])
                if ((difff(image[neighbor][1],image[current_position][1]) < min) \
                                     and (neighbor not in visited) and (image[neighbor][1] > image[neighbor][2])):
                    # if image[neighbor][1] < image[neighbor][2]:
                    #     #print('nastavio')
                    #     continue
                    next = neighbor
                    min = difff(image[neighbor][1],image[current_position][1])
                    print(next,next in visited)
                    #if image[next][1] < image[next][2]:
                        #print('jeste')
                    #print(next,neighbors)


            if next is None:
                next_position = max_y(visited)
                # print('max y',next_position,next_position in visited)
                # print()
                # for _ in get_neighbors(next_position):
                #     print(_ in visited,end=' ')
                # print()
                
                #print('max',next_position,image[next_position][1],image[next_position][2])
                
                
                    
            else:
                next_position = next
                #print(next_position,image[next_position][1],image[next_position][2])

            #if image[next_position][1] < 2 and image[next_position][0] < 2 and image[next_position][2] < 2: 
            global b
            if image[next_position][1] < image[next_position][2]: 
                    b += 1
                else:
                    b = 0
            
                
            if b==20:
                print('')
                green_found = False

            
            path.append(next_position)
            visited.append(next_position)
            current_position = next_position
        print(briter)    
        return path

def are_elements_unique(lst):
    print(len(set(lst)))
    return len(lst) == len(set(lst))

segmentation_result = np.zeros_like(image)
result = segmentation()
print(are_elements_unique(result))
for position in result:
    segmentation_result[position] = image[position]

cv2.imshow("Result", segmentation_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

IndentationError: unindent does not match any outer indentation level (<string>, line 82)

In [38]:
import cv2
import numpy as np
image = cv2.imread("img/1s.png")
path = []
visited = []
def segment():
        height, width,_ = image.shape
        for i in range(height):
            for j in range(width):
                if image[i,j][1] > 150:# image[i,j][2]:
                    path.append((i,j))
        return path

segmentation_result = np.zeros_like(image)
result = segment()
for position in result:
    segmentation_result[position] = image[position]

cv2.imshow("Result", segmentation_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [515]:
import cv2

def get_pixel_rgb(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Get the RGB value of the clicked pixel
        b, g, r = image[y, x]  # OpenCV uses BGR format
        print(f"RGB value at ({x}, {y}): R={r}, G={g}, B={b}")

# Load the image
image_path = 'img/1s.png'
image = cv2.imread(image_path)

# Display the image and set the mouse callback function
cv2.imshow('Image', image)
cv2.setMouseCallback('Image', get_pixel_rgb)

# Wait for the user to click a pixel
cv2.waitKey(0)

# Close the OpenCV window
cv2.destroyAllWindows()


RGB value at (292, 178): R=121, G=81, B=63
RGB value at (323, 109): R=132, G=94, B=73
RGB value at (243, 221): R=100, G=58, B=41
RGB value at (363, 287): R=116, G=68, B=54
RGB value at (314, 170): R=137, G=92, B=71
RGB value at (62, 218): R=0, G=41, B=2
RGB value at (44, 149): R=0, G=95, B=53
RGB value at (28, 65): R=0, G=30, B=11
RGB value at (59, 231): R=0, G=52, B=26
RGB value at (74, 277): R=2, G=125, B=51
RGB value at (113, 223): R=112, G=74, B=59
RGB value at (147, 104): R=117, G=81, B=67


In [ ]:
l=[1,2,3,4,5]
print(l[-6])